# Build Ukk23 examples/rfmip-clear-sky

*2024-07-18*

Based on: <https://github.com/peterukk/rte-rrtmgp-nn>

Ukkonen, P., & Hogan, R. J. (2023). Implementation of a machine-learned gas optics parameterization in the ECMWF Integrated Forecasting System: RRTMGP-NN 2.0. Geoscientific Model Development, 16(11), 3241–3261. <https://doi.org/10.5194/gmd-16-3241-2023>

Instructions: examples/rfmip-clear-sky/Readme.md

In [1]:
%cd build

/home/x/Documents/inpe/Ukk23/ukk23test01/build


In [2]:
%env FC=gfortran
%env FCFLAGS=-ffree-line-length-none -m64 -march=native -O3
%env NCHOME=/usr
%env NFHOME=/usr
%env BLASLIB=openblas

env: FC=gfortran
env: FCFLAGS=-ffree-line-length-none -m64 -march=native -O3
env: NCHOME=/usr
env: NFHOME=/usr
env: BLASLIB=openblas


## Build the RTE+RRTMGP libraries

In [3]:
! make clean

rm -f *.optrpt *.mod *.o librrtmgp.a librte.a libneural.a


In [4]:
! make

gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/mo_rte_kind.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/mo_rte_rrtmgp_config.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/mo_rte_util_array.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/kernels/mo_optical_props_kernels.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/mo_optical_props.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/mo_source_functions.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/kernels/mo_fluxes_broadband_kernels.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/mo_fluxes.F90
gfortran -ffree-line-length-none -m64 -march=native -O3 -I/usr/include -c ../rte/kernels/mo_rte_solver_kernels.F90
gfortran -ffree-line-length-none -m64 

## Build the executables (examples)

In [5]:
%cd ../examples/rfmip-clear-sky

/home/x/Documents/inpe/Ukk23/ukk23test01/examples/rfmip-clear-sky


In [6]:
! make

VAR="../../"
gfortran -ffree-line-length-none -m64 -march=native -O3 -I../..//build -I/usr/include -c ../mo_simple_netcdf.F90 -fopenmp
gfortran -ffree-line-length-none -m64 -march=native -O3 -I../..//build -I/usr/include -c mo_rfmip_io.F90 -fopenmp
gfortran -ffree-line-length-none -m64 -march=native -O3 -I../..//build -I/usr/include -c ../mo_load_coefficients.F90 -fopenmp
gfortran -ffree-line-length-none -m64 -march=native -O3 -I../..//build -I/usr/include -c rrtmgp_rfmip_lw.F90 -fopenmp
gfortran -ffree-line-length-none -m64 -march=native -O3 -o rrtmgp_rfmip_lw rrtmgp_rfmip_lw.o mo_simple_netcdf.o mo_rfmip_io.o mo_load_coefficients.o ../..//build/librte.a ../..//build/librrtmgp.a ../..//build/libneural.a -L../..//build -L/usr/lib -L/usr/lib -lrrtmgp -lrte -lneural -lnetcdff -lnetcdf -lopenblas  -fopenmp
gfortran -ffree-line-length-none -m64 -march=native -O3 -I../..//build -I/usr/include -c rrtmgp_rfmip_sw.F90 -fopenmp
gfortran -ffree-line-length-none -m64 -march=native -O3 -o rrtmgp_r

## Run via run-rfmip-examples.py

In [7]:
! python3 run-rfmip-examples.py

Running ./rrtmgp_rfmip_lw   8 multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc ../../rrtmgp/data/rrtmgp-data-lw-g256-2018-12-04.nc
 Usage: rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)] [physics_index (1,2)]  input_output file]
 OR:  rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] [forcing_index] [physics_index] [NN_lw_abs_file] [NN_lw_planck_file]
 input file:multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                     
 nexp:          18 ncol:         100 nlay:          60 block_size:           8
 Doing          225 blocks of size            8
 Calculation uses RFMIP gases: h2o co2 o3 n2o co ch4 o2 n2 ccl4 cfc11 cfc12 cfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 setting no2 to zero
 starting clear-sky LW computations, using lookup-table as RRTMGP kernel
 Elapsed time on everything   0.555999994    
 --------------------------------------

## Run manually

In [9]:
%%bash
./rrtmgp_rfmip_lw \
    8 \
    multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
    ../../rrtmgp/data/rrtmgp-data-lw-g256-2018-12-04.nc \
    1 \
    1

 Usage: rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)] [physics_index (1,2)]  input_output file]
 OR:  rrtmgp_rfmip_lw [block_size] [rfmip_file] [k-distribution_file] [forcing_index] [physics_index] [NN_lw_abs_file] [NN_lw_planck_file]
 input file:multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc                                                                     
 nexp:          18 ncol:         100 nlay:          60 block_size:           8
 Doing          225 blocks of size            8
 Calculation uses RFMIP gases: h2o co2 o3 n2o co ch4 o2 n2 ccl4 cfc11 cfc12 cfc22 hfc143a hfc125 hfc23 hfc32 hfc134a cf4 no2 
 setting no2 to zero
 starting clear-sky LW computations, using lookup-table as RRTMGP kernel
 Elapsed time on everything   0.629999995    
 -----------------------------------------------------------------------------------------
 mean of flux_down is:   103.235573    
 mean of flux_up is:   301.596527    
 ---------------

In [10]:
! ./rrtmgp_rfmip_sw \
    8 \
    multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc \
    ../../rrtmgp/data/rrtmgp-data-sw-g224-2018-12-04.nc \
    1 \
    1

 Usage: rrtmgp_rfmip_sw [block_size] [rfmip_file] [k-distribution_file] [forcing_index (1,2,3)]
 OR:  rrtmgp_rfmip_sw [block_size] [rfmip_file] [k-distribution_file] [forcing_index] [NN_sw_abs_file] [NN_sw_ray_file]
STOP provide 1-4 or 6 arguments
